# 基础思路（baseline）

## 步骤1：更新或安装所需环境

In [1]:
!pip install --upgrade modelscope requests urllib3 tqdm pandas
!apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip






## 步骤2：下载数据集

In [2]:
!axel -n 12 -a https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv

Initializing download: https://ai-contest-static.xfyun.cn/2024/%E5%A4%A7%E6%A8%A1%E5%9E%8B%E8%83%BD%E5%8A%9B%E8%AF%84%E6%B5%8B%EF%BC%9A%E4%B8%AD%E6%96%87%E6%88%90%E8%AF%AD%E9%87%8A%E4%B9%89%E4%B8%8E%E8%A7%A3%E6%9E%90%E6%8C%91%E6%88%98%E8%B5%9B/test_input.csv
ERROR 403: Forbidden.


## 步骤3：构建模型（使用Qwen2-7B-Instruct）

In [3]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-7B-Instruct")

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.24it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


## 步骤4：读取数据集

In [4]:
import pandas as pd
test = pd.read_csv('./test_input.csv', header=None)

In [5]:
# 查看数据集大小
print(f"数据集的大小为: {test.shape[0]}\n前50条数据如下：\n")

# 查看前50条赛事数据集（赛题要求根据每行句子，给出5个可能匹配的成语）
for test_prompt in test[0].values[:50]:
    print(test_prompt)

数据集的大小为: 2972
前50条数据如下：

双方心往一处想，意志坚定。
两端都极为艰难，难以作出决定。
雄浑深远的意旨。细腻微妙。高超巧妙。
描述水流湍急，且快速且深远。
避免被引诱做出不道德或可疑的行为。
肘部肩窝。比喻事物发生于身边。
他张开嘴巴，吞咽着口水。
仅见一面，不足以深入了解。
向四处散发施舍，同时手中拿着碗盆。
比喻把装备脱下，放下武器。
对于任何战役，都要一败涂地。
阴谋真相已经昭然于众。
比喻因为无能为力而写下了好文章。
古代文人士大夫经常举行诗歌朗诵会。
不能形容为不高兴，只能说明没劲儿。
表现得举止端庄，很有教养。
1.洗刷兵器2.喂养战马3.准备作战
关注生命垂危者，关怀濒危者。
比喻面对挑战，坚韧不拔地前行。
过去的科举考试中被选拔为进士的称号。
相似程度极高或相差无几。
比喻不断地补充、堆砌和延伸。
以安逸快乐的生活和劳动为重。
她仍然每天教导她的儿子。
犹以火为耕，比喻原始、简朴的农耕方式。
指困境中处于不利地位。
搜集和研究其内在道理。
没有任何人帮助和支持。
这位作者的文章风格与自己非常相似。
国力强大，军事力量已经停止。
相互勾结维持；相互利用。
比喻进程飞快，日行千里。
只有一个目的，追求利润。
务必谨慎对待，慎重处理事务。
无法言表，只能感慨万千。
形容气势磅礴的文章风格。
根据贡献的大小给予奖励。
让国家蒙羞，民众蒙难。
形容有权势的人极其残忍和无礼。
坚决要求再次向某人强调。
采取措施；采取办法；采取行动；实行
心中充满了疑惑，还没有找到解答。
累累罪行，遍历无穷。形容罪恶极重。
形容人的容貌清爽俊雅，风度翩翩。
辞掉本职工作去做其他的事情。
秦汉时期，勋位至高者都佩戴金印和紫绶。
创立独特的风格，与众不同。
到处都是冰雪覆盖的环境，形容严冬天气。
旧事物被废弃；为了新事物而采取措施。
①向人行礼。②用作哀悼词或祭奠语。


## 步骤5：输出成语

In [ ]:
from tqdm import tqdm
import os
import re

i = 1
# 假设 test 是一个 DataFrame
# 遍历测试数据集的第一项的值，目的是生成与给定句子最相关的五个成语
for test_prompt in tqdm(test[0].values, total=len(test[0].values), desc="处理进度"):
    i = i + 1
    # 构造提示信息，要求模型输出与句子最相关的五个成语
    prompt = f"把下面句子总结为五个成语，写在一行中：{test_prompt}"

    # 初始化一个长度为5的列表，填充默认成语“同舟共济”
    words = ['同舟共济'] * 5

    # 构建聊天消息格式，用于提示模型进行生成
    messages = [
        {"role": "user", "content": prompt},
    ]
    # 应用聊天模板对消息进行处理，准备模型输入
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # 对输入文本进行编码，准备模型输入数据
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # 生成回答，限制最大生成长度
    generated_ids = model.generate(
        model_inputs.input_ids,
        num_beams=3,# 5 已经超显存了
        no_repeat_ngram_size=2,
        num_return_sequences=3,
        do_sample=False,
        length_penalty=1.0,
        
    )
    # 提取模型输出，去除输入部分
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    # 解码模型输出，去除特殊标记
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 清理回答文本，确保格式统一
    response = response.replace('\n', ' ').replace('、', ' ')
    # 提取回答中的成语，确保每个成语长度为4且非空
    words =  response # [x for x in response.split() if len(x) == 4 and x.strip() != '']
    
    

    # 如果生成的成语列表长度不满足要求（即20个字符），则使用默认成语列表
    #if len(' '.join(words).strip()) != 24:
        #words = ['同舟共济'] * 5

    # 将最终的成语列表写入提交文件
    with open('submit.csv', 'a+', encoding='utf-8') as up:
        up.write(' '.join(words) + '\n')

    
    # 查看阶段性结果
    if i % 50 == 0:
        tqdm.write(f"大模型第{i}次返回的结果是：\n   {response}\n")
        tqdm.write(f"submit.cvs第{i}行输出结果：\n   {words}\n")
    
    # 为了尽快拿到结果，我们暂时仅获得500个结果（如果有时间的话，可以删除这两行）
    if i == 10:
        break

print('submit.csv 已生成')

处理进度:   0%|          | 0/2972 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
处理进度:   0%|          | 5/2972 [01:46<17:29:05, 21.22s/i